# Div Arb Straight Up

**The Goal of this notebook is to locate areas of "straight up arbitrage" where we can gather at least .5% 2-day profit on liquid options**

*We Define "Straight Up Arb" as *Arbitrage from purchasing a put close to expiry where*     
`Div*(1-tax) - Cost_option < Underlying - Strike`   
We define percentage of arbitrage as The Percentage of Arbitrage that we can get from this as   
`Div * (1 - tax) / (Underlying + Cost_Option)`   
Since we can put on a position daily we would be satisfied with anything like a .2% return (keep in mind that our profits only get compounded every 1.5 months since there is a 1.5 month delay before getting payment from the dividend. In fact we may slowly bleed liquidity until dividend payments start coming through).

In [ ]:
Dividend*(1-tax)/(Strike+Cost)